First initial libraries. Machine learning has been left out for now.

Additionally, the datasets are much too large for my laptops memory, so I will load truncated versions. I will also keep a commented-out cell which attempts to either load the whole dataset, or use a SQL server for it.

In [2]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import nltk
import math
from sqlalchemy import create_engine
#nltk.download()

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

The first cell here will upload smaller sections of the datasets.

In [3]:

raw_2016 = pd.read_csv('properties_2016.csv', nrows = 30000)
raw_2017 = pd.read_csv('properties_2017.csv', nrows = 30000)
vals_2016 = pd.read_csv('train_2016_v2.csv')
vals_2017 = pd.read_csv('train_2017.csv')


The next cell will load the entire datasets, and I will likely not ever be able to use this cell.

In [4]:
#raw_2016 = pd.read_csv('properties_2016.csv', low_memory = False)
#raw_2017 = pd.read_csv('properties_2017.csv', low_memory = False)
#vals_2016 = pd.read_csv('train_2016_v2.csv')
#vals_2017 = pd.read_csv('train_2017.csv')

Finally a third possible solution that lets me use SQL to pull in large datasets after I have figured out how to trim the dataset.

This still has big memory issues, so it may not end up being used either. I would also like to know if there is a way to minimize a cell as this the SQL cell takes up a lot of room.

In [5]:
'''
vals_2016 = pd.read_csv('train_2016_v2.csv')
vals_2017 = pd.read_csv('train_2017.csv')

csv_database = create_engine('sqlite:///csv_database.db')

# Next we load a the 2016 dataset into a SQL engine table chunk by chunk
chunksize = 10000
i_1 = 0
j_1 = 1
for df in pd.read_csv('properties_2016.csv', chunksize=chunksize, iterator=True):
      df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
      df.index += j_1
      i_1+=1
      df.to_sql('table_2016', csv_database, if_exists='append')
      j = df.index[-1] + 1
#now we do the same for 2017 into another table
i_2 = 0
j_2 = 1
for df in pd.read_csv('properties_2017.csv', chunksize=chunksize, iterator=True):
      df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
      df.index += j_2
      i_2+=1
      df.to_sql('table_2017', csv_database, if_exists='append')
      j = df.index[-1] + 1
        
df = pd.read_sql_query('SELECT COl1 table_2017', csv_database)
df.head()
'''
print()

Before we can merge the dataframes with their vals we need to see how long each are. We note that there are far more in the vals dataframe, but since we did not truncate those csv files but did for the raw files, we do not know if vals is complete.

In [12]:
len(raw_2016),len(vals_2016)

(30000, 90275)

Now we will merge the raw df with the vals df. However, we note that the df's when merged on 'inner' are MUCH smaller than the starting ones, meaning that the train csv's were smaller than the raw csv's. However, since this notebook is primary concerned with munging and cleaning, we will proceed anyway, hoping for a solution where we can deal with the larger datasets.

Another important observation is that the mergeing process gained us two extra rows so we no longer have unique entries in parcelid. We will need to look at how into that.

In [17]:
df2016 = pd.merge(raw_2016, vals_2016, how='inner', left_on='parcelid', right_on='parcelid')
df2017 = pd.merge(raw_2017, vals_2017, how='inner', left_on='parcelid', right_on='parcelid')
len(df2016),len(df2016['parcelid'].unique())

(896, 894)

Now lets get some information on the structure of these dataframes as well as what types of data are inside it. The biggest observations we can make here is that there are many empty columns, likely resulting from the extreme truncation. As a result, we will need merge with how='left'

In [18]:
df2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 896 entries, 0 to 895
Data columns (total 60 columns):
parcelid                        896 non-null int64
airconditioningtypeid           276 non-null float64
architecturalstyletypeid        0 non-null float64
basementsqft                    0 non-null float64
bathroomcnt                     896 non-null float64
bedroomcnt                      896 non-null float64
buildingclasstypeid             0 non-null float64
buildingqualitytypeid           551 non-null float64
calculatedbathnbr               884 non-null float64
decktypeid                      7 non-null float64
finishedfloor1squarefeet        67 non-null float64
calculatedfinishedsquarefeet    890 non-null float64
finishedsquarefeet12            851 non-null float64
finishedsquarefeet13            0 non-null float64
finishedsquarefeet15            35 non-null float64
finishedsquarefeet50            67 non-null float64
finishedsquarefeet6             4 non-null float64
fips       